<a href="https://colab.research.google.com/github/Saurabh08281997/NLP/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! kaggle competitions download -c fake-news

 54% 25.0M/46.5M [00:00<00:00, 102MB/s] 
100% 46.5M/46.5M [00:00<00:00, 139MB/s]


In [5]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import pandas as pd

In [7]:
df=pd.read_csv('train.csv')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.shape

(20800, 5)

In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
###Drop Nan Values
df=df.dropna()


In [13]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [15]:
## Get the Dependent features
y=df['label']

In [16]:
X.shape

(18285, 4)

In [17]:
y.shape

(18285,)

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'2.8.2'

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [21]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [22]:
messages=X.copy()

In [23]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [44]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [45]:
messages.reset_index(inplace=True)

In [47]:
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [48]:
import nltk
import re
from nltk.corpus import stopwords

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [53]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [54]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [55]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2502, 3386, 2498, 2334, 415, 2233, 1826, 1982, 2563, 2337],
 [3287, 3984, 2404, 3248, 3004, 826, 4144],
 [4010, 4578, 2332, 4838],
 [595, 3059, 3613, 3642, 4793, 4659],
 [510, 3004, 285, 2414, 4781, 3839, 3004, 3184, 1686, 1873],
 [251,
  2348,
  2312,
  1255,
  612,
  204,
  3706,
  4401,
  1220,
  3762,
  4530,
  2574,
  134,
  3950,
  4144],
 [2224, 1920, 4750, 1867, 135, 2868, 1517, 4723, 137, 1109, 2787],
 [1673, 4354, 195, 2310, 3983, 1911, 204, 1983, 137, 1109, 2787],
 [1812, 3344, 3685, 129, 3377, 1964, 1189, 2424, 204, 1273],
 [4294, 833, 3245, 3729, 3141, 180, 2929, 2671],
 [2156, 4861, 1429, 2, 3026, 1387, 1032, 501, 3475, 4802, 1976],
 [3642, 1838, 415, 1964, 204, 3983],
 [3779, 2129, 3967, 4209, 4839, 919, 3968, 2959, 2508],
 [73, 1391, 4271, 3295, 4374, 3299, 4399, 137, 1109, 2787],
 [2381, 3573, 786, 2685, 2906, 137, 1109, 2787],
 [2290, 1192, 4314, 4017, 4979, 4025, 1306, 3342, 2589, 3875],
 [1235, 3652, 3984],
 [1838, 3126, 1800, 592, 204, 4893, 140, 4144],
 [3652, 1

In [56]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [57]:
onehot_repr[1]

[3287, 3984, 2404, 3248, 3004, 826, 4144]

### Embedding Representation

In [58]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2502 3386 2498 ...    0    0    0]
 [3287 3984 2404 ...    0    0    0]
 [4010 4578 2332 ...    0    0    0]
 ...
 [ 610 3550 4755 ...    0    0    0]
 [1454 3983 1933 ...    0    0    0]
 [ 431  601 2492 ...    0    0    0]]


In [59]:
embedded_docs[1]

array([3287, 3984, 2404, 3248, 3004,  826, 4144,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [60]:
embedded_docs[0]

array([2502, 3386, 2498, 2334,  415, 2233, 1826, 1982, 2563, 2337,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [61]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
len(embedded_docs),y.shape

(18285, (18285,))

In [63]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [64]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [74]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 9ms/step - loss: 0.3314 - accuracy: 0.8304 - val_loss: 0.2133 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1485 - accuracy: 0.9420 - val_loss: 0.1912 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1111 - accuracy: 0.9603 - val_loss: 0.2097 - val_accuracy: 0.9226
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0877 - accuracy: 0.9695 - val_loss: 0.2474 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0658 - accuracy: 0.9780 - val_loss: 0.2474 - val_accuracy: 0.9162
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0530 - accuracy: 0.9820 - val_loss: 0.2940 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0440 - accuracy: 0.9847 - val_loss: 0.3524 - val_accuracy: 0.9158
Epoch 

### Adding Dropout 

In [73]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [75]:
y_pred=model.predict(X_test)

In [76]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [77]:
from sklearn.metrics import confusion_matrix

In [78]:
confusion_matrix(y_test,y_pred)

array([[3127,  292],
       [ 232, 2384]])

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9131731565865783

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

